In [1]:
import numpy as np
import pandas as pd

import pandas_profiling

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore') 
#warnings.filterwarnings(action='ignore', category=FutureWarning, module='xgboost')

from category_encoders import OneHotEncoder, OrdinalEncoder, TargetEncoder

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier

import eli5
from eli5.sklearn import PermutationImportance

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from pdpbox import pdp
from pdpbox.pdp import pdp_isolate, pdp_plot
from pdpbox.pdp import pdp_interact, pdp_interact_plot
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('Gold1.csv')

df

,datetime,US_interest_rate,Gold_Price,Gold_Change,Silver_Price,Silver_Change,US_10_year_rate,10_year_rate_Change,DollarIndex,DollarIndex_Change,USD_KRW_Price,USD_KRW_Change
0,2021-04-26,1.1,1776.80,0.12,25.990,-0.02,1.5648,0.12,90.757,-0.09,1115.13,0.04
1,2021-04-25,1.1,1774.65,-0.13,25.995,-0.31,1.5630,0.22,90.839,-0.53,1115.13,0.04
2,2021-04-23,1.1,1777.00,-0.24,26.075,-0.40,1.5595,1.16,90.839,-0.53,1114.72,-0.36
3,2021-04-22,1.1,1781.20,-0.62,26.180,-1.47,1.5416,-1.12,91.321,0.21,1118.79,0.22
4,2021-04-21,1.1,1792.30,0.84,26.570,2.83,1.5590,-0.45,91.129,-0.11,1116.35,-0.08
...,...,...,...,...,...,...,...,...,...,...,...,...
7964,1990-01-08,8.1,403.00,-0.81,5.244,-1.39,8.0260,1.17,92.800,-0.88,683.60,-0.01
7965,1990-01-05,8.1,406.30,1.93,5.318,0.95,8.0260,0.00,93.620,0.49,683.70,-0.07
7966,1990-01-04,8.1,398.60,1.17,5.268,2.27,8.0260,0.00,93.160,-2.16,684.20,0.10
7967,1990-01-03,8.1,394.00,-1.40,5.151,-0.62,8.0260,0.00,95.220,0.02,683.50,0.31


In [3]:
# leakage 제거

df = df.drop('Gold_Change' , axis = 1)
df = df.drop('Silver_Price' , axis = 1)
df = df.drop('Silver_Change' , axis = 1)
df = df.drop('USD_KRW_Price' , axis = 1)
df = df.drop('USD_KRW_Change' , axis = 1)

In [4]:
# test 랑 train 으로 나누기
X_test = df[:1594]
X_train = df[1594:]

In [5]:
#train을 또 나누기 val로
X_val = X_train[5100:]
X_train = X_train[:5100]


print(X_test.shape)
print(X_train.shape)
print(X_val.shape)

(1594, 7)
(5100, 7)
(1275, 7)


In [6]:
X_train = X_train.sort_values(by = 'datetime')
X_test = X_test.sort_values(by = 'datetime')
X_val = X_val.sort_values(by = 'datetime')

In [7]:
X_train = X_train.drop('datetime', axis = 1)
X_test = X_test.drop('datetime', axis = 1)
X_val = X_val.drop('datetime', axis = 1)


In [8]:
target = 'Gold_Price'

features = X_train.drop(columns=[target]).columns

y_train = X_train[target]
X_train = X_train[features]
y_val = X_val[target]
X_val = X_val[features]
y_test = X_test[target]
X_test = X_test[features]

In [9]:
pipe = Pipeline([
    ('xgb',XGBRegressor(eval_metric = 'rmse'
                  , random_state=2
                  , n_jobs=-1
                 ))])

param_grid = {'xgb__n_estimators': [100, 200, 300, 500, 600, 700, 800], 
              'xgb__learning_rate': [0.01, 0.05, 0.1, 0.15,  0.2],
              'xgb__max_depth' : [1, 3, 5, 7, 10],
              'xgb__sub_sample' : [ 0.2, 0.5, 0.8, 1],
              'xgb__colsample_bytree': [0.2, 0.5, 0.8, 1]}

randomizedsearch = RandomizedSearchCV(pipe, param_grid, cv=4, scoring ='neg_root_mean_squared_error', n_jobs=-1, verbose=4, random_state = 42)
randomizedsearch.fit(X_train, y_train)

print('Best Parameter :', randomizedsearch.best_params_)
print('Best Score : ', -(randomizedsearch.best_score_))

model = randomizedsearch.best_estimator_
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

# print('\n훈련 RMSE: ', sqrt(mean_squared_error(y_train, y_pred_train)))
print('검증 RMSE: ', np.sqrt(mean_squared_error(y_val, y_pred_val)))

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[10:49:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "sub_sample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Best Parameter : {'xgb__sub_sample': 0.2, 'xgb__n_estimators': 700, 'xgb__max_depth': 1, 'xgb__learning_rate': 0.05, 'xgb__colsample_bytree': 0.5}
Best Score :  287.0960498116991
검증 RMSE:  97.04080079943856


In [10]:
from sklearn.metrics import r2_score

print('훈련 정확도: ', model.score(X_train, y_train))
print('검증 정확도: ', model.score(X_val, y_val))
print('검증 r2_score 정확도: ', r2_score(y_pred_val, y_val))

훈련 정확도:  0.9742060034834885
검증 정확도:  -20.488761592209332
검증 r2_score 정확도:  -0.40003716632174324


In [12]:
from scipy.stats import randint, uniform

pipe = make_pipeline(
    TargetEncoder(), 
    SimpleImputer(), 
    RandomForestRegressor(random_state=2)
)

dists = {
    'targetencoder__smoothing': [2.,20.,50.,60.,100.,500.,1000.], # int로 넣으면 error(bug)
    'targetencoder__min_samples_leaf': randint(1, 10),     
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestregressor__n_estimators': randint(50, 500), 
    'randomforestregressor__max_depth': [5, 10, 15, 20, None], 
    'randomforestregressor__max_features': uniform(0, 1) # max_features
}

clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='neg_mean_absolute_error',  
    verbose=1,
    n_jobs=-1
)

clf.fit(X_train, y_train);



Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [13]:
print('훈련 정확도: ', clf.score(X_train, y_train))
print('검증 정확도: ', clf.score(X_val, y_val))

훈련 정확도:  -9.62124279180944
검증 정확도:  -43.47872866688579
